In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *


StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 4, Finished, Available, Finished)

In [3]:
%%sql
create table if not exists ECommerce_Lakehouse.gold_product
(
    Product_ID	Long	,
Product_Category	string	,
Product	string	,
Created_TS	timestamp,
Modified_TS timestamp
)
using delta

StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from ECommerce_Lakehouse.gold_product").first()[0]

StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 6, Finished, Available, Finished)

In [5]:
df_bronze=spark.sql("select distinct Product_Category, Product from ECommerce_Lakehouse.bronze_sales where Modified_TS>'{}'".format(Max_Date))

StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 7, Finished, Available, Finished)

In [6]:
Max_ID=spark.sql("select coalesce(max(Product_ID),0) from ECommerce_Lakehouse.gold_product").first()[0]

StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 8, Finished, Available, Finished)

In [7]:
df=df_bronze.withColumn("Product_ID",monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 9, Finished, Available, Finished)

In [8]:
df.createOrReplaceTempView("ViewProduct")

StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 10, Finished, Available, Finished)

In [9]:
%%sql
merge into ECommerce_Lakehouse.gold_product as gp 
using ViewProduct as vp  
on gp.Product=vp.Product and gp.Product_Category=vp.Product_Category
when MATCHED then 
update set
gp.Modified_TS= current_timestamp()

when not matched then
INSERT
(
gp.Product_ID,
gp.Product_Category,
gp.Product,
gp.Created_TS,
gp.Modified_TS
)
VALUES
(
vp.Product_ID,
vp.Product_Category,
vp.Product,
current_timestamp(),
current_timestamp()

)



StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 11, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [10]:
%%sql

SELECT * FROM gold_product

StatementMeta(, 86b5b030-7996-4b95-b5b9-26e8edcbced6, 12, Finished, Available, Finished)

<Spark SQL result set with 42 rows and 5 fields>